# Повторяю ноутбук Валерия Бабушкина про снижение дисперсии с помощью стратификации

In [1]:
import numpy as np
from tqdm import tqdm
from scipy.stats import norm

In [25]:
def show_diff_se(sample_size: int, n_iter: int = 100):
    
    # Истинные размеры страт в генеральной совокупности
    strat_1_len = 10 ** 7
    strat_2_len = 10 ** 6
    
    # Список попадание СКО в доверительный интервал для каждой итерации
    in_ci_acc = []
    
    for _ in tqdm(range(n_iter)):
        # Общий размер генеральной совокупности
        len_gen = strat_1_len + strat_2_len
        # Страты ген. совокупности
        gen_strat_1 = np.random.normal(100, 10, size=strat_1_len)
        gen_strat_2 = np.random.normal(50, 5, size=strat_2_len)
        # Генеральная совокупность
        gen = np.concatenate([gen_strat_1, gen_strat_2])
        gen_mean = np.mean(gen)
        
        # Размер сэмпла из генеральной совокупности для оценки весов страт
        valuation_sample_size = int(0.1 * len_gen)
        # Участники эксперимента до эксперимента
        idx = np.random.choice(range(len(gen)), size=valuation_sample_size, replace=False)
        strat_1_before = gen[idx[idx <= strat_1_len]]
        strat_2_before = gen[idx[idx > strat_1_len]]
        # Участники эксперимента после экспериента
        idx = np.random.choice(range(len(gen)), size=valuation_sample_size, replace=False)
        strat_1_after = gen[idx[idx <= strat_1_len]]
        strat_2_after = gen[idx[idx > strat_1_len]]
        exp_after = np.concatenate([strat_1_after, strat_2_after])
        # веса страт до эксперимента. Важно! именно ДО эксперимента
        w1 = strat_1_before.shape[0] / valuation_sample_size
        w2 = strat_2_before.shape[0] / valuation_sample_size
        
        # сэмплирую участников эксперимента после эксперимента
        idx = np.random.choice(range(len(exp_after)), size=sample_size, replace=False)
        strat_1_after_samples = exp_after[idx[idx <= len(strat_1_after)]]
        strat_2_after_samples = exp_after[idx[idx > len(strat_1_after)]]
    
        strat_se = ((strat_1_after_samples.var() * w1 + strat_2_after_samples.var() * w2) / sample_size) ** 0.5
        strat_mean = strat_1_after_samples.mean() * w1 + strat_2_after_samples.mean() * w2
        
        # z-score для 95% доверительного интервала (двустороннего) ~1.96
        z_score = norm.ppf(1 - 0.05 / 2)
        in_ci = 1 if (gen_mean >= strat_mean - z_score * strat_se) and (gen_mean <= strat_mean + z_score * strat_se) else 0
        in_ci_acc.append(in_ci)
        
    
    print(f"Доля попаданий стратифицированого СКО в доверительный интервал: {sum(in_ci_acc) / n_iter}")
    print(f"Стратифицированное среднее: {strat_mean}")
    print(f"Стратифицированное СКО: {strat_se}")
    
    # Оценка СКО через бутстрап
    exp_b = np.random.choice(exp_after, size=(sample_size, 1000), replace=True)
    std_b = np.std(np.mean(exp_b, axis=0))
    print(f"СКО через bootstrap: {std_b}")

In [29]:
show_diff_se(1000, 100)

100%|█████████████████████████████████████████| 100/100 [06:12<00:00,  3.72s/it]

Доля попаданий стратифицированого СКО в доверительный интервал: 0.96
Стратифицированное среднее: 95.28660270267949
Стратифицированное СКО: 0.31009440393100385
СКО через bootstrap: 0.5242678279698993


In [28]:
show_diff_se(1000, 10)

100%|███████████████████████████████████████████| 10/10 [00:35<00:00,  3.59s/it]

Доля попаданий стратифицированого СКО в доверительный интервал: 0.9
Стратифицированное среднее: 95.76268927763243
Стратифицированное СКО: 0.3129756305418904
СКО через bootstrap: 0.5541631321072553


## Вариант, который был в видео
https://www.youtube.com/watch?v=F-6CXV7r1C0

In [2]:
# Генеральная совокупность
strat_1 = np.random.normal(100, 10, 10**6)
strat_2 = np.random.normal(50, 5, 10**6)
gen_len = strat_1.shape[0] + strat_2.shape[0]
gen_mean = np.concatenate([strat_1, strat_2]).mean()

In [9]:
# Кол-во наблюдений из страт
len_1 = 1000
len_2 = 1000
# Веса страт
w1 = len_1 / (len_1 + len_2)
w2 = len_2 / (len_1 + len_2)

std_strats = []
mean_strats = []
in_ci_acc = []

for i in tqdm(range(1000)):
    sample_1 = np.random.choice(strat_1, len_1, replace=False)
    sample_2 = np.random.choice(strat_2, len_2, replace=False)
    
    std_strat = ((sample_1.var() * w1 + sample_2.var() * w2) / (len_1 + len_2)) ** 0.5
    mean_strat = sample_1.mean() * w1 + sample_2.mean() * w2
    std_strats.append(std_strat)
    mean_strats.append(mean_strat)
    
    # Вхождение в доверительный интервал
    z_score = norm.ppf(1 - 0.05 / 2)
    in_ci = 1 if (gen_mean >= mean_strat - z_score * std_strat) and (gen_mean <= mean_strat + z_score * std_strat) \
                else 0
    in_ci_acc.append(in_ci)

print(f"Доля попаданий в доверительный интервал: {np.mean(in_ci_acc)}")
print(f"Стратифицированное СКО: {np.mean(std_strats)}")
print(f"Стратифицированное среднее: {np.mean(mean_strats)}")
print(f"Обычное СКО: {np.concatenate([sample_1, sample_2]).std()}")
print(f"Обычное среднее: {np.concatenate([sample_1, sample_2]).mean()}")

100%|███████████████████████████████████████| 1000/1000 [01:04<00:00, 15.58it/s]

Доля попаданий в доверительный интервал: 0.958
Стратифицированное СКО: 0.17677775714253977
Стратифицированное среднее: 75.00793218000591
Обычное СКО: 26.005521729724556
Обычное среднее: 74.97589228314999


In [10]:
# Проверка СКО и среднего через бутстрап
b = np.random.choice(np.concatenate([strat_1, strat_2]), size=((len_1+len_2), 1000), replace=True)
b_mean = b.mean(axis=0)
print(f"СКО бутстрепа: {b_mean.std()}")
print(f"Среднее бутстрепа: {b_mean.mean()}")

СКО бутстрепа: 0.6000222247838686
Среднее бутстрепа: 74.98789167523742


In [ ]:
# Остались вопросы:
# 1. Почему обычное СКО в несколько раз отличается от СКО бутстрепа?
# 2. Как уменьшенная через стратификацию дисперсия поможет нам проводить эксперименты быстрее?
# Ведь от того, что мы иначе оценили дисперсию, истинная дисперсия не изменилась.